In [84]:
#import necessary packages
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import time
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, f_classif, chi2, f_regression

In [20]:
player_ratings_19 = pd.read_csv('Data/2019-20/Players/player_ratings_19_a_09.csv')
features_19 = pd.read_csv('Data/2019-20/Fixtures/PL19_20_features.csv')
player_ratings_20 = pd.read_csv('Data/2020-21/Players/player_ratings_a_09.csv')
features_20 = pd.read_csv('Data/2020-21/Fixtures/PL20_21_features.csv')
player_ratings_21 = pd.read_csv('Data/2021-22/Players/player_ratings_21_a_09.csv')
features_21 = pd.read_csv('Data/2021-22/Fixtures/PL21_22_features.csv')
player_ratings_20.replace(['Brighton & Hove Albion','Manchester United', 'Newcastle United', 'Sheffield United', 'Tottenham Hotspur', 'West Bromwich Albion', 'West Ham United','Wolverhampton Wanderers'], ['Brighton', 'Manchester Utd','Newcastle Utd', 'Sheffield Utd', 'Tottenham','West Brom' ,'West Ham', 'Wolves'], inplace=True)
player_ratings_20['game_home_ID'] = player_ratings_20['HomeTeam'] + player_ratings_20['Date']
player_ratings_20['game_away_ID'] = player_ratings_20['AwayTeam'] + player_ratings_20['Date']
player_ratings_20

,Unnamed: 0,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,game_home_ID,game_away_ID,...,rating_a_gk,rating_a_df,rating_a_mf,rating_a_att,rating_a_bench,influence_a,creativity_a,threat_a,bps_a,selected_a
0,0,2020-09-12,12:30,Fulham,Arsenal,0,3,A,Fulham2020-09-12,Arsenal2020-09-12,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
1,1,2020-09-12,15:00,Crystal Palace,Southampton,1,0,H,Crystal Palace2020-09-12,Southampton2020-09-12,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
2,2,2020-09-12,17:30,Liverpool,Leeds United,4,3,H,Liverpool2020-09-12,Leeds United2020-09-12,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
3,3,2020-09-12,20:00,West Ham,Newcastle Utd,0,2,A,West Ham2020-09-12,Newcastle Utd2020-09-12,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
4,4,2020-09-13,14:00,West Brom,Leicester City,0,3,A,West Brom2020-09-13,Leicester City2020-09-13,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375,2021-05-23,16:00,Liverpool,Crystal Palace,2,0,H,Liverpool2021-05-23,Crystal Palace2021-05-23,...,18.095864,77.508335,34.222302,114.179672,23.624494,141.292815,147.381546,121.639973,114.763385,2.700597e+06
376,376,2021-05-23,16:00,Manchester City,Everton,5,0,H,Manchester City2021-05-23,Everton2021-05-23,...,54.939201,99.089373,99.931465,83.451864,18.141813,214.701344,167.231180,133.112182,201.690987,6.039664e+06
377,377,2021-05-23,16:00,Sheffield Utd,Burnley,1,0,H,Sheffield Utd2021-05-23,Burnley2021-05-23,...,13.723130,83.687464,85.162467,30.281391,10.164714,113.227836,130.831265,117.996689,99.825233,1.289402e+06
378,378,2021-05-23,16:00,West Ham,Southampton,3,0,H,West Ham2021-05-23,Southampton2021-05-23,...,52.776573,97.320803,99.866236,19.887791,108.619953,112.338538,97.568369,71.328588,78.067443,3.597889e+06


In [23]:
PL_2019_total =  pd.merge(features_19, player_ratings_19, on= 'game_home_ID', how='left')
PL_2020_total =  pd.merge(features_20, player_ratings_20, on= 'game_home_ID', how='left')
PL_2021_total =  pd.merge(features_21, player_ratings_21, on= 'game_home_ID', how='left')
frames = [PL_2019_total, PL_2020_total, PL_2021_total]
PL_19_20_21_all_features = pd.concat(frames)
PL_19_20_21_all_features

,Unnamed: 0_x,Div,Date_x,Time_x,HomeTeam_x,AwayTeam_x,FTHG_x,FTAG_x,FTR_x,HTHG,...,rating_a_gk,rating_a_df,rating_a_mf,rating_a_att,rating_a_bench,influence_a,creativity_a,threat_a,bps_a,selected_a
0,0,E0,2019-08-09,20:00,Liverpool,Norwich City,4,1,H,4,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
1,1,E0,2019-08-10,12:30,West Ham,Manchester City,0,5,A,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
2,2,E0,2019-08-10,15:00,Bournemouth,Sheffield Utd,1,1,D,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
3,3,E0,2019-08-10,15:00,Burnley,Southampton,3,0,H,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
4,4,E0,2019-08-10,15:00,Crystal Palace,Everton,0,0,D,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375,E0,2022-05-22,16:00,Crystal Palace,Manchester Utd,1,0,H,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
376,376,E0,2022-05-22,16:00,Leicester City,Southampton,4,1,H,0,...,30.426248,90.718082,38.302624,27.739725,4.715052,184.049913,35.385130,50.186897,128.755329,8.253030e+05
377,377,E0,2022-05-22,16:00,Liverpool,Wolves,3,1,H,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
378,378,E0,2022-05-22,16:00,Manchester City,Aston Villa,3,2,H,0,...,0.000000,138.256940,193.946287,15.334034,9.397336,202.612485,230.250828,198.143528,148.016855,5.879654e+06


In [36]:
PL_19_20_21_all_features['gk_difference'] = PL_19_20_21_all_features['rating_h_gk'] - PL_19_20_21_all_features['rating_a_gk']
PL_19_20_21_all_features['df_difference'] = PL_19_20_21_all_features['rating_h_df'] - PL_19_20_21_all_features['rating_a_df']
PL_19_20_21_all_features['mf_difference'] = PL_19_20_21_all_features['rating_h_mf'] - PL_19_20_21_all_features['rating_a_mf']
PL_19_20_21_all_features['att_difference'] = PL_19_20_21_all_features['rating_h_att'] - PL_19_20_21_all_features['rating_a_att']
PL_19_20_21_all_features['bench_difference'] = PL_19_20_21_all_features['rating_h_bench'] - PL_19_20_21_all_features['rating_a_bench']
PL_19_20_21_all_features['influence_difference'] = PL_19_20_21_all_features['influence_h'] - PL_19_20_21_all_features['influence_a']
PL_19_20_21_all_features['creativity_difference'] = PL_19_20_21_all_features['creativity_h'] - PL_19_20_21_all_features['creativity_a']
PL_19_20_21_all_features['threat_difference'] = PL_19_20_21_all_features['threat_h'] - PL_19_20_21_all_features['threat_a']
PL_19_20_21_all_features['bps_difference'] = PL_19_20_21_all_features['bps_h'] - PL_19_20_21_all_features['bps_a']
PL_19_20_21_all_features['selected_difference'] = PL_19_20_21_all_features['selected_h'] - PL_19_20_21_all_features['selected_a']
PL_19_20_21_all_features['overall_h'] = PL_19_20_21_all_features['rating_h_gk'] + PL_19_20_21_all_features['rating_h_df'] + PL_19_20_21_all_features['rating_h_mf'] + PL_19_20_21_all_features['rating_h_att'] + PL_19_20_21_all_features['rating_h_bench']
PL_19_20_21_all_features['overall_a'] = PL_19_20_21_all_features['rating_a_gk'] + PL_19_20_21_all_features['rating_a_df'] + PL_19_20_21_all_features['rating_a_mf'] + PL_19_20_21_all_features['rating_a_att'] + PL_19_20_21_all_features['rating_a_bench']
PL_19_20_21_all_features['overall_difference'] = PL_19_20_21_all_features['overall_h'] - PL_19_20_21_all_features['overall_a']
PL_19_20_21_all_features

,Unnamed: 0_x,Div,Date_x,Time_x,HomeTeam_x,AwayTeam_x,FTHG_x,FTAG_x,FTR_x,HTHG,...,att_difference,bench_difference,influence_difference,creativity_difference,threat_difference,bps_difference,selected_difference,overall_h,overall_a,overall_difference
0,0,E0,2019-08-09,20:00,Liverpool,Norwich City,4,1,H,4,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
1,1,E0,2019-08-10,12:30,West Ham,Manchester City,0,5,A,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
2,2,E0,2019-08-10,15:00,Bournemouth,Sheffield Utd,1,1,D,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
3,3,E0,2019-08-10,15:00,Burnley,Southampton,3,0,H,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
4,4,E0,2019-08-10,15:00,Crystal Palace,Everton,0,0,D,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375,E0,2022-05-22,16:00,Crystal Palace,Manchester Utd,1,0,H,1,...,17.004112,42.803780,78.183687,31.288983,50.563073,67.829308,2.930965e+06,179.443592,0.000000,179.443592
376,376,E0,2022-05-22,16:00,Leicester City,Southampton,4,1,H,0,...,29.145907,23.096406,24.274186,8.288232,-30.099457,10.599418,4.178858e+06,263.589242,191.901731,71.687511
377,377,E0,2022-05-22,16:00,Liverpool,Wolves,3,1,H,1,...,26.900002,60.880212,117.585669,32.251976,86.203308,96.274694,3.902984e+06,246.156609,0.000000,246.156609
378,378,E0,2022-05-22,16:00,Manchester City,Aston Villa,3,2,H,0,...,25.620656,55.885047,-56.131097,-25.603259,48.868307,-16.173457,4.021852e+06,444.529338,356.934597,87.594741


In [78]:
#prepare Data for ML Algos, including all features from benchmark + rating differences
#add column for H A X
conditions = [
    (PL_19_20_21_all_features['FTR_x'] == 'H'),
    (PL_19_20_21_all_features['FTR_x'] == 'A'),
    (PL_19_20_21_all_features['FTR_x'] == 'D')]
choices = [0, 1, 2]
PL_19_20_21_all_features['identifier'] = np.select(conditions, choices)
#PL20_21_features.drop(PL20_21_features[ PL20_21_features.Wk < 6].index, inplace=True)
y = PL_19_20_21_all_features['identifier'].to_numpy()
feature_names_1 = ['Rating difference', 'Rating_Home', 'Rating_Away', 'ewmH', 'ewmA', 'ewmDifference', 'Elo_h_before','Elo_a_before', 'elo_difference', 'goalsH', 'goalsA', 'goalsDifference', 'shotsH', 'shotsA','shotsDifference', 'cornersH', 'cornersA', 'cornersDifference', 'formH', 'formA', 'form_difference']
feature_names_2 = ['gk_difference', 'df_difference', 'mf_difference', 'att_difference', 'bench_difference','overall_difference' ,'influence_difference', 'creativity_difference', 'threat_difference', 'bps_difference', 'selected_difference', 'overall_h', 'overall_a']
feature_names_3 = ['rating_h_gk', 'Rating difference', 'Rating_Home', 'Rating_Away', 'ewmH', 'ewmA', 'ewmDifference', 'Elo_h_before','Elo_a_before', 'elo_difference', 'goalsH', 'goalsA', 'goalsDifference', 'shotsH', 'shotsA','shotsDifference', 'cornersH', 'cornersA', 'cornersDifference', 'formH', 'formA', 'form_difference', 'rating_a_gk', 'rating_h_df', 'rating_a_df', 'rating_h_mf', 'rating_a_mf', 'rating_h_att', 'rating_a_att', 'rating_h_bench', 'rating_a_bench', 'influence_h', 'influence_a', 'creativity_h', 'creativity_a', 'threat_h', 'threat_a', 'bps_h', 'bps_a', 'selected_h', 'selected_a', 'gk_difference', 'df_difference', 'mf_difference', 'att_difference', 'bench_difference','overall_difference' ,'influence_difference', 'creativity_difference', 'threat_difference', 'bps_difference', 'selected_difference', 'overall_h', 'overall_a']
X_1 = PL_19_20_21_all_features[feature_names_1].to_numpy()
X_2 = PL_19_20_21_all_features[feature_names_2].to_numpy()
X_3 = PL_19_20_21_all_features[feature_names_3].to_numpy()

In [79]:
#split Data in training and test set and train model
X_train, X_test, y_train, y_test = train_test_split(X_1, y, test_size=0.2, random_state=42)
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

RandomForestClassifier()

In [80]:
rf_model = RandomForestClassifier()
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y, test_size=0.2, random_state=42)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y, test_size=0.2, random_state=42)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y, test_size=0.2, random_state=42)
rf_model.fit(X_train_1, y_train_1)
rf_model.fit(X_train_2, y_train_2)
rf_model.fit(X_train_3, y_train_3)
#start_time = time.time()
# importances = rf_model.feature_importances_
# std = np.std([tree.feature_importances_ for tree in rf_model.estimators_], axis=0)
# #elapsed_time = time.time() - start_time
#
#
# #feature_names = [f"feature {i}" for i in range(X.shape[1])]
#
#
# forest_importances = pd.Series(importances, index=feature_names_1)
# forest_importances = forest_importances.sort_values()
# fig, ax = plt.subplots()
# fig.tight_layout(pad=-15, w_pad=0.5, h_pad=-2)
#
# forest_importances.plot.barh(ax=ax, yerr=std)
# ax.set_title("Feature importances using MDI")
# ax.set_xlabel("Mean decrease in impurity")
print(rf_model.score(X_test_3, y_test_3))

0.49122807017543857


In [77]:
score_rf_1 = []
score_rf_2 = []
score_rf_3 = []
score_xg_1 = []
score_xg_2 = []
score_xg_3 = []
i = 100
rf_model_1 = RandomForestClassifier()
rf_model_2 = RandomForestClassifier()
rf_model_3 = RandomForestClassifier()
xgb_model_1 = xgb.XGBClassifier(seed=82)
xgb_model_2 = xgb.XGBClassifier(seed=82)
xgb_model_3 = xgb.XGBClassifier(seed=82)
for i in range(1, i):
    X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y, test_size=0.2, random_state=42)
    X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y, test_size=0.2, random_state=42)
    X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y, test_size=0.2, random_state=42)
    rf_model_1.fit(X_train_1, y_train_1)
    rf_model_2.fit(X_train_2, y_train_2)
    rf_model_3.fit(X_train_3, y_train_3)
    xgb_model_1.fit(X_train_1, y_train_1)
    xgb_model_2.fit(X_train_2, y_train_2)
    xgb_model_3.fit(X_train_3, y_train_3)
    score_rf_1.append(rf_model_1.score(X_test_1, y_test_1))
    score_rf_2.append(rf_model_2.score(X_test_2, y_test_2))
    score_rf_3.append(rf_model_3.score(X_test_3, y_test_3))
    score_xg_1.append(xgb_model_1.score(X_test_1, y_test_1))
    score_xg_2.append(xgb_model_2.score(X_test_2, y_test_2))
    score_xg_3.append(xgb_model_3.score(X_test_3, y_test_3))
print('RF1:', np.mean(score_rf_1))
print('RF2:',np.mean(score_rf_2))
print('RF3:',np.mean(score_rf_3))
print('XG1:',np.mean(score_xg_1))
print('XG2:',np.mean(score_xg_2))
print('XG3:',np.mean(score_xg_3))

RF1: 0.48759525075314547
RF2: 0.4783802941697678
RF3: 0.48467127414495836
XG1: 0.45175438596491235
XG2: 0.4736842105263158
XG3: 0.4736842105263158


In [83]:
print(rf_model_3.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [82]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [85]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train_3, y_train_3)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [86]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

In [87]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

    return accuracy
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train_3, y_train_3)
base_accuracy = evaluate(base_model, X_test_3, y_test_3)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test_3, y_test_3)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Model Performance
Average Error: 0.6890 degrees.
Accuracy = nan%.
Model Performance
Average Error: 0.6637 degrees.
Accuracy = -inf%.
Improvement of nan%.


C:\Users\giuli\AppData\Local\Temp\ipykernel_20928\1957789784.py:4: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * np.mean(errors / test_labels)
C:\Users\giuli\AppData\Local\Temp\ipykernel_20928\1957789784.py:4: RuntimeWarning: invalid value encountered in true_divide
  mape = 100 * np.mean(errors / test_labels)
C:\Users\giuli\AppData\Local\Temp\ipykernel_20928\1957789784.py:4: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * np.mean(errors / test_labels)
